In [5]:
### 08_create_spatial_presence_any_extreme

import rasterio
import numpy as np
import os

# -----------------------------
# Config
# -----------------------------
years = range(2000, 2021)

stack_templates = {
    "any": "../data/derived/annual_stacks/binary/annual_stack_any_{}.tif",
    "extreme": "../data/derived/annual_stacks/binary/annual_stack_extreme_{}.tif"
}

out_base = "../data/derived/annual_stacks/binary/spatial_presence"

bands = {
    "wf": 1,
    "bt": 2,
    "hd": 3,
    "pd": 4
}

# Only the combinations you requested
combos = {
    "wf_bt": ("wf", "bt"),
    "wf_hd": ("wf", "hd"),
    "wf_pd": ("wf", "pd"),
    "bt_hd": ("bt", "hd"),
    "bt_pd": ("bt", "pd"),
    "wf_bt_hd": ("wf", "bt", "hd"),
    "wf_bt_pd": ("wf", "bt", "pd"),
}

nodata_val = 0
pixel_area_km2 = (30 * 30) / 1e6

# -----------------------------
# Processing
# -----------------------------
for mode, template in stack_templates.items():
    print(f"\n==============================")
    print(f"Processing mode: {mode.upper()}")
    print(f"==============================")

    out_dir = f"{out_base}/{mode}"
    os.makedirs(out_dir, exist_ok=True)

    collapsed = {}
    meta = None

    # ---- Singles ----
    for name, band in bands.items():
        print(f"\nCollapsing {name} ({mode}) ...")
        arr_out = None

        for yr in years:
            f = template.format(yr)
            print(f"  Processing {f}")

            with rasterio.open(f) as src:
                arr = src.read(band).astype(np.uint8)

                if arr_out is None:
                    arr_out = arr.copy()
                    meta = src.meta.copy()
                    meta.update(
                        dtype=rasterio.uint8,
                        count=1,
                        compress='DEFLATE',
                        nodata=nodata_val,
                        tiled=True,
                        BIGTIFF='YES'
                    )
                else:
                    arr_out = ((arr_out > 0) | (arr > 0)).astype(np.uint8)

        out = f"{out_dir}/{name}_{mode}_presence.tif"
        with rasterio.open(out, "w", **meta) as dst:
            dst.write(arr_out, 1)

        px = np.count_nonzero(arr_out)
        area = px * pixel_area_km2

        print(f"  → {out}")
        print(f"    pixels: {px:,}")
        print(f"    area (km²): {area:,.2f}")

        collapsed[name] = arr_out

    # ---- Combinations ----
    print(f"\nCreating combinations ({mode})...")

    for cname, keys in combos.items():
        print(f"  Creating {cname}_{mode}_presence")

        combo = np.ones_like(collapsed[keys[0]], dtype=bool)
        for k in keys:
            combo &= (collapsed[k] > 0)

        combo = combo.astype(np.uint8)

        out = f"{out_dir}/{cname}_{mode}_presence.tif"
        with rasterio.open(out, "w", **meta) as dst:
            dst.write(combo, 1)

        px = np.count_nonzero(combo)
        area = px * pixel_area_km2

        print(f"  → {out}")
        print(f"    pixels: {px:,}")
        print(f"    area (km²): {area:,.2f}")

print("\n✅ ANY and EXTREME spatial presence rasters created with `_presence` suffix.")




Processing mode: ANY

Collapsing wf (any) ...
  Processing ../data/derived/annual_stacks/binary/annual_stack_any_2000.tif
  Processing ../data/derived/annual_stacks/binary/annual_stack_any_2001.tif
  Processing ../data/derived/annual_stacks/binary/annual_stack_any_2002.tif
  Processing ../data/derived/annual_stacks/binary/annual_stack_any_2003.tif
  Processing ../data/derived/annual_stacks/binary/annual_stack_any_2004.tif
  Processing ../data/derived/annual_stacks/binary/annual_stack_any_2005.tif
  Processing ../data/derived/annual_stacks/binary/annual_stack_any_2006.tif
  Processing ../data/derived/annual_stacks/binary/annual_stack_any_2007.tif
  Processing ../data/derived/annual_stacks/binary/annual_stack_any_2008.tif
  Processing ../data/derived/annual_stacks/binary/annual_stack_any_2009.tif
  Processing ../data/derived/annual_stacks/binary/annual_stack_any_2010.tif
  Processing ../data/derived/annual_stacks/binary/annual_stack_any_2011.tif
  Processing ../data/derived/annual_stack